In [1]:
import os
import sys
import pandas
import numpy

import findspark
findspark.init("/usr/local/spark/spark")

import pyspark
from pyspark.sql.window import Window
import pyspark.sql.functions as func

from pyspark.rdd import reduce
from pyspark.sql.types import DoubleType
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import GaussianMixture, KMeansModel, KMeans
from pyspark.ml.linalg import SparseVector, VectorUDT, Vector, Vectors

In [2]:
from pyspark.sql.functions import udf, col, struct
from pyspark.sql.types import ArrayType, DoubleType, StringType
from pyspark.mllib.linalg.distributed import RowMatrix, DenseMatrix

In [3]:
conf = pyspark.SparkConf().setMaster("local[*]").set("spark.driver.memory", "4G").set("spark.executor.memory", "4G")
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)

In [4]:
cluster_files =  "/Users/simondi/PROJECTS/target_infect_x_project/src/tix-analysis/data/outlier-removal"

In [258]:
df = spark.read.parquet(cluster_files)

In [259]:
def n_param_gmm(k, p, n):
    n_mean = k * p
    n_var = k * p*(p + 1) / 2
    n_mix = k - 1
    return n_mean + n_var + n_mix

In [260]:
def split_features(data):
    def to_array(col):
        def to_array_(v):
            return v.toArray().tolist()

        return udf(to_array_, ArrayType(DoubleType()))(col)

    len_vec = len(data.select("features").take(1)[0][0])
    data = (data.withColumn("f", to_array(col("features")))
            .select(["prediction"]+  [col("f")[i] for i in range(len_vec)]))

    for i, x in enumerate(data.columns):
        if x.startswith("f["):
            data = data.withColumnRenamed(
                x, x.replace("[", "_").replace("]", ""))

    return data

In [303]:
def n_param_kmm(k, p):
    n_mean = k * p
    n_var = 1
    return n_mean + n_var

In [273]:
n = d_k.count()
p = len(d_k.take(1)[0]) - 1

In [307]:
max_k_model = KMeans(k=50, seed=1).fit(df)
transformed_data = split_features(max_k_model.transform(df))

In [308]:
transformed_data.take(1)

[Row(prediction=13, f_0=0.983076443174413, f_1=0.40052619964395375, f_2=0.059322697068162594, f_3=0.8912970547151504, f_4=0.6459296915260802, f_5=-1.6407744126709187, f_6=0.0779898194213411, f_7=0.6963730444812934, f_8=0.20419738806418344, f_9=-0.6405384795826222, f_10=-1.2687766832708212, f_11=-0.5029967514166027, f_12=0.4632598970025907, f_13=-1.3307274422272615, f_14=-0.44745668769115393)]

In [293]:
def compute_variance(K, data, model, p):
    total_var = 0
    ni = scipy.zeros(K)
    p = len(data.take(1)[0]) - 1

    for i in range(K):
        data_cluster = data.filter("prediction==" + str(i)).drop("prediction")
        ni[i] =  data_cluster.count()
        rdd = data_cluster.rdd.map(list)
        means = model.clusterCenters()[i]
        var = (RowMatrix(rdd).rows
                   .map(lambda x: (x - means).T.dot(x - means))
                   .reduce(lambda x, y: x + y))
        total_var += var
    total_var /= ((data.count() - K) * p)

    return total_var, ni

In [310]:
compute_variance(50, transformed_data, max_k_model, p)

(0.17133309276888586,
 array([ 2.,  1.,  1.,  7.,  4.,  1.,  1.,  1.,  1.,  3.,  1.,  3.,  1.,
         1.,  1.,  2.,  3.,  1.,  1.,  2.,  3.,  2.,  1.,  2.,  1.,  2.,
         1.,  1.,  2.,  1.,  2.,  1.,  2.,  5.,  1.,  3.,  4.,  2.,  2.,
         1.,  1.,  2.,  1.,  1.,  2.,  1.,  3.,  3.,  2.,  1.]))

In [316]:
def loglik(data, K, model):
    n = data.count()
    p = len(data.take(1)[0]) - 1
    total_var, ni = compute_variance(K, data, model, p)
    ll = 0
    
    for i in range(K):
        l = ni[i] * scipy.log(ni[i])
        l -= ni[i] * scipy.log(n) 
        l -= .5 * ni[i] * p * scipy.log(2 * scipy.pi * total_var) 
        l -= .5 * (ni[i] - 1) * p
        ll += l
    bic = ll - .5 * scipy.log(n) * (p + 1) * K
    
    return ll, bic, total_var, ni

In [292]:
max_loglik = loglik(transformed_data, 10, n, max_k_model)

P 15


In [356]:
def lrt(max_loglik, loglik, max_params, params):
    t = 2 * (max_loglik - loglik)
    df = max_params - params    
    if df == 0:
        return 1
    p_val = 1 - scipy.stats.chi2.cdf(t, df=df)
    return p_val

In [326]:
def get_model(k, data):
    mod = KMeans(k=k, seed=23).fit(data)
    transformed_data = split_features(mod.transform(data))
    ll, bic, var, ni = loglik(transformed_data, k, mod)
    n_params = n_param_kmm(k, p)
    return (ll, bic, var, ni, n_params)

In [360]:
Kmod = get_model(30, df)
for i in range(2, 30 + 1):
    smmod = get_model(i, df)
    l_rt = lrt(Kmod[0], smmod[0], Kmod[-1], smmod[-1])
    print(i, Kmod[0], smmod[0], Kmod[-1], smmod[-1], l_rt)

2 -982.280166978 -1683.95070031 451 31 0.0
3 -982.280166978 -1543.70976091 451 46 0.0
4 -982.280166978 -1478.75699244 451 61 0.0
5 -982.280166978 -1437.0254444 451 76 0.0
6 -982.280166978 -1414.98059385 451 91 0.0
7 -982.280166978 -1390.12382265 451 106 0.0
8 -982.280166978 -1359.26309058 451 121 0.0
9 -982.280166978 -1334.94689332 451 136 0.0
10 -982.280166978 -1302.32049625 451 151 0.0
11 -982.280166978 -1269.4160301 451 166 0.0
12 -982.280166978 -1258.85717206 451 181 0.0
13 -982.280166978 -1234.42234948 451 196 0.0
14 -982.280166978 -1202.19684998 451 211 6.41708908233e-14
15 -982.280166978 -1208.42803672 451 226 0.0
16 -982.280166978 -1211.75472954 451 241 0.0
17 -982.280166978 -1154.21972143 451 256 2.5294932815e-10
18 -982.280166978 -1180.1557268 451 271 0.0
19 -982.280166978 -1115.92464953 451 286 7.96985200058e-07
20 -982.280166978 -1096.67007164 451 301 3.60716691906e-05
21 -982.280166978 -1134.87842255 451 316 3.44169137634e-15
22 -982.280166978 -1083.12357993 451 331 4.4300

In [358]:
def bs(p):
    lefts, rights, mids = [], [], []
    left, right = 2, 30
    mid = int((left + right) / 2)
    lrts = []
    
    K_model = get_model(right, df)
    while True:
        mids.append(mid)
        lefts.append(left)
        rights.append(right)
        
        m_model = get_model(mid, df)    
    
        l_rt = lrt(K_model[0], m_model[0], K_model[-1], m_model[-1])
        print(mid, right, K_model[0], m_model[0], K_model[-1], m_model[-1], l_rt)
        lrts.append(l_rt)
        
        if l_rt > p:
            mid, right= int((left + mid) / 2), mid
        elif l_rt < p:
            mid, left = int((right + mid) / 2), mid
        if left == lefts[-1] and right == rights[-1]:
            break
            
    return right

i = bs(.05)

16 30 -982.280166978 -1211.75472954 451 241 0.0
23 30 -982.280166978 -1113.4627652 451 346 1.88737914186e-15
26 30 -982.280166978 -1034.77334483 451 391 0.00029440380176
28 30 -982.280166978 -975.854110772 451 421 1.0
27 28 -982.280166978 -1010.68615375 451 406 0.111380825565
26 27 -982.280166978 -1034.77334483 451 391 0.00029440380176
